In [8]:
# %pip install kokoro soundfile

# %pip install misaki[ja]
# %pip install lingua-language-detector
# %pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 22.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import re
import json
import soundfile as sf
from kokoro import KPipeline
import torch.nn as nn


from transformers import pipeline

from lingua import Language, LanguageDetectorBuilder

detector = LanguageDetectorBuilder.from_all_languages().build()

translator = None

rnn = nn.RNN(input_size=..., hidden_size=..., num_layers=2, dropout=0.2)


def translate_eng_to_jp(eng_text:str):
    global translator
    if translator is None:
        translator = pipeline("translation_en_to_ja", model="Helsinki-NLP/opus-mt-en-jap")
    result = translator(eng_text)
    jp_text = result[0]['translation_text']
    print(jp_text)
    return jp_text.strip()

def process_transcript_file(
    transcript_file,
    output_folder="audios",
    output_json="metadata.json",
    voice="af_heart",      # Change voice as needed (e.g. 'af_heart' for an American English voice)
    speed=1,
    lang_code="j"         
):
    # 🇺🇸 'a' => American English,
    # 🇬🇧 'b' => British English
    # 🇯🇵 'j' => Japanese: pip install misaki[ja]
    # 🇨🇳 'z' => Mandarin Chinese: pip install misaki[zh]
    # 🇪🇸 'e' => Spanish es
    # 🇫🇷 'f' => French fr-fr
    # 🇮🇳 'h' => Hindi hi
    # 🇮🇹 'i' => Italian it
    # 🇧🇷 'p' => Brazilian Portuguese pt-br
 
    os.makedirs(output_folder, exist_ok=True)
    
    
    pipeline = KPipeline(lang_code=lang_code)
    
    metadata = []
    
    with open(transcript_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
 
    pattern = re.compile(r"\[(.*?)\]\s*(.+)")

    
    for idx, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
        
        match = pattern.match(line)
        if not match:
            print(f"Skipping line {idx}: Format not recognized: {line}")
            continue
        
        timestamp, text = match.groups()
        detected_language = detector.detect_language_of(text)

        if detected_language == Language.ENGLISH:
            jp_text = translate_eng_to_jp(text)
            text = jp_text
        else:
            raise ValueError(f"Not supported language: {detected_language}")
            
        print(f"Text: {text} \n\n")
 
        try:
            generator = pipeline(text, voice=voice, speed=speed, split_pattern=r'\n+')
            gs, ps, audio = next(generator)

       
        except Exception as e:
            print(f"Error generating audio for segment {idx}: {e}")
            continue
   
        audio_path = os.path.join(output_folder, f"audio_{idx}.wav")
        
        try:
 
            sf.write(audio_path, audio, 24000)
            print(f"Saved audio for segment {idx} to {audio_path}")
        except Exception as e:
            print(f"Error saving audio for segment {idx}: {e}")
            audio_path = None
 
        metadata.append({
            "timestamp": timestamp,
            "text": text,
            "audio_path": audio_path
        })
    
 
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=4)
    
    print(f"Metadata saved to {output_json}")

 

In [ ]:
process_transcript_file("transcripts/transcript-timestamp.txt", output_folder= "jp_audios")
 

/home/hm3/miniconda3/envs/transformer/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/home/hm3/miniconda3/envs/transformer/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/hm3/miniconda3/envs/transformer/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


これ が 要求 し た こと で , われわれ は , 彼 ら の 支配 者 たち の 響き を , 波 で は な い か .
Text: これ が 要求 し た こと で , われわれ は , 彼 ら の 支配 者 たち の 響き を , 波 で は な い か . 


Saved audio for segment 0 to jp_audios/audio_0.wav
そして ,  時 に 沿 っ た 力 に よ っ て , 彼 ら の 支配 力 に 満ち て い る よう に と , 書 い て あ っ た の で あ る .
Text: そして ,  時 に 沿 っ た 力 に よ っ て , 彼 ら の 支配 力 に 満ち て い る よう に と , 書 い て あ っ た の で あ る . 


Saved audio for segment 1 to jp_audios/audio_1.wav
こう し て , ふたり の 者 が 前 に も 聞きいれ な かっ た の で ,
Text: こう し て , ふたり の 者 が 前 に も 聞きいれ な かっ た の で , 


Saved audio for segment 2 to jp_audios/audio_2.wav
しかし , 知 ら な い 者 たち は そこ に 走 っ て 行 く .
Text: しかし , 知 ら な い 者 たち は そこ に 走 っ て 行 く . 


Saved audio for segment 3 to jp_audios/audio_3.wav
と い う の は , いわば , 波 に と っ て や す べ き こと で あ る .
Text: と い う の は , いわば , 波 に と っ て や す べ き こと で あ る . 


Saved audio for segment 4 to jp_audios/audio_4.wav
あなた は 一 種 の 重 さ を 取 り , 残り の 民 を 雇 う の に , 少し だけ で も , たわん だっ た 者 を も 取 っ て その まま に し て お き なさ い .
Text: あなた は 一 種 の 重 さ を 取 り , 残り の 民 を 雇 う の に , 少し